In [1]:
%pip install loguru==0.7.3 python-dotenv==1.0.1 PyYAML==6.0.2 torch==2.5.1 tqdm==4.67.1 typer==0.15.1 matplotlib==3.10.0 pyarrow==18.1.0 setuptools==75.1.0 protobuf==4.25.3 ultralytics==8.3.84 ray==2.43.0 albumentations==2.0.5 pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
import locale
from eyeinthesky.config import Config
import os
import pandas as pd
import gc
import torch
from ultralytics import YOLO
from datetime import datetime
import json

sys.dont_write_bytecode = True
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
# Load config

config = Config.load("../config/config.yaml")
print(config)

{'train': {'model': 'yolo12n', 'project': 'EyeInTheSky', 'data': 'VisDrone.yaml', 'task': 'detect', 'device': 0, 'epochs': 1, 'batch': 16, 'workers': 8, 'seed': 42, 'plots': True, 'imgsz': 640, 'exist_ok': False, 'save': True, 'save_period': 10, 'val': True, 'warmup_epochs': 10, 'visualize': True, 'show': True, 'single_cls': False, 'rect': False, 'cos_lr': False, 'resume': False, 'amp': True, 'fraction': 1.0, 'freeze': 'None', 'cache': False}, 'model': 'yolo12n.pt', 'project': 'EyeInTheSky', 'dataset_name': 'VisDrone', 'raw_data_dir': 'data/raw', 'data_dir': 'data', 'config_dir': 'config', 'interim_data_dir': 'data/interim', 'processed_data_dir': 'data/processed', 'external_data_dir': 'data/external', 'models_dir': 'models', 'reports_dir': 'reports', 'figures_dir': 'reports/figures', 'val': {'project': 'EyeInTheSky', 'name': 'YOLOv12-VisDrone-Validation', 'half': True, 'conf': 0.25, 'iou': 0.6, 'split': 'test', 'rect': True, 'plots': True}, 'tune': {'name': 'YOLOv12-VisDrone-Tuning', '

In [4]:
# Dataset path

from pathlib import Path

current_path = Path.cwd().parent.absolute()
dataset_path = os.path.join(current_path, "config", f"{config['dataset_name']}.yaml")
print(dataset_path)

/ext/home/fperagine/EyeInTheSky/config/VisDrone.yaml


In [5]:
# Wandb

# wandb_api_key = Config.get_wandb_key(Path("../.env"))

In [6]:
# Load top 5 tune results by fitness

csv_path = "../data/processed/wandb_export_2025-03-05T10_24_46.923+01_00.csv"
df = pd.read_csv(csv_path)
df = df.rename(columns={'Name': 'name'})

df['fitness'] = df['metrics/mAP50(B)'] * 0.1 + df['metrics/mAP50-95(B)'] * 0.9
df_sorted = df.sort_values(by='fitness', ascending=False).head(5)

columns_to_show = ['name', 'fitness', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)',
                  'metrics/precision(B)', 'metrics/recall(B)', 'optimizer',
                  'lr0', 'lrf', 'momentum', 'weight_decay', 'cos_lr', 'imgsz', 'box', 'cls', 'dfl']

top_5 = df_sorted[columns_to_show].reset_index(drop=True)
print("Top 5 Models by Fitness Score:")
display(top_5)

Top 5 Models by Fitness Score:


,name,fitness,metrics/mAP50(B),metrics/mAP50-95(B),metrics/precision(B),metrics/recall(B),optimizer,lr0,lrf,momentum,weight_decay,cos_lr,imgsz,box,cls,dfl
0,_tune_8290d_00015,0.184317,0.30057,0.17140,0.42157,0.30252,AdamW,0.001,0.10,0.8,0.001,True,640.0,3.872225,0.947276,4.117353
1,_tune_9e260_00006,0.183925,0.29899,0.17114,0.39727,0.30826,AdamW,0.001,0.10,0.8,0.000,True,640.0,5.120607,0.816281,4.608205
2,_tune_8290d_00008,0.183161,0.29657,0.17056,0.39720,0.30612,AdamW,0.001,0.01,0.8,0.000,True,640.0,3.427384,0.759476,3.358324
3,_tune_8290d_00012,0.182955,0.29973,0.16998,0.42051,0.30218,AdamW,0.001,0.01,0.8,0.000,True,640.0,7.015643,1.946285,4.618466
4,_tune_9e260_00015,0.182941,0.29761,0.17020,0.40803,0.30691,AdamW,0.001,0.10,0.8,0.000,True,640.0,8.909576,0.940505,0.985378


In [7]:
# Clear cache after each run

def clear_cache():
    # Clear CUDA cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Clear Python garbage collector
    gc.collect()

In [8]:
# Store results

def save_results(dir, name, results):
    os.makedirs(dir, exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_path = f"{dir}/{name}_{timestamp}.json"

    with open(results_path, 'w') as f:
        json.dump(results, f, indent=4, default=str)
    
    print(f"Results saved to {results_path}")

In [9]:
# # Load model

model_path = os.path.join(current_path, config['models_dir'], f"{config['model']}")
model = YOLO(model_path)
print(model_path)

/ext/home/fperagine/EyeInTheSky/models/yolo12n.pt


In [10]:
from ultralytics.data.dataset import YOLODataset
import numpy as np
from ultralytics.utils import LOGGER, colorstr
from pathlib import Path

class VisDroneDataset(YOLODataset):
    """
    Custom dataset for VisDrone that merges 'pedestrian' (class 0) and 'people' (class 1) into 'person'.
    All other classes are shifted down by 1 to account for this merge.
    """
    
    def __init__(self, *args, **kwargs):
        """Initialize with standard YOLODataset behavior."""
        super().__init__(*args, **kwargs)
        
        # Update the class names in the data dictionary
        if self.data and "names" in self.data:
            # Create new names dictionary with merged classes
            new_names = {0: "person"}  # Merged class (pedestrian + people)
            # Add remaining classes with shifted indices
            for old_idx, name in self.data["names"].items():
                if old_idx >= 2:  # Skip the merged classes
                    new_names[old_idx - 1] = name
            
            # Update the data dictionary
            self.data["names"] = new_names
            self.data["nc"] = len(new_names)
            
            LOGGER.info(f"{colorstr('VisDroneDataset:')} Using merged classes: {self.data['names']}")
    
    def get_labels(self):
        """
        Override to remap class indices immediately after labels are loaded.
        """
        # Get the original labels using the parent method
        labels = super().get_labels()
        
        # Track statistics for logging
        count_class_0 = 0
        count_class_1 = 0
        count_shifted = 0
        
        # Now remap all class indices to our merged class structure
        for label in labels:
            if "cls" in label:
                cls = label["cls"]
                
                # Count stats before remapping
                count_class_0 += np.sum(cls == 0)
                count_class_1 += np.sum(cls == 1)
                
                # Convert class 1 to class 0 (merge people into pedestrian)
                mask_class_1 = cls == 1
                if np.any(mask_class_1):
                    cls[mask_class_1] = 0
                    
                # Shift classes > 1 down by 1
                mask_greater_than_1 = cls > 1
                count_shifted += np.sum(mask_greater_than_1)
                if np.any(mask_greater_than_1):
                    cls[mask_greater_than_1] -= 1
        
        LOGGER.info(f"{colorstr('VisDroneDataset:')} Remapped {count_class_1} 'people' instances to 'person'")
        LOGGER.info(f"{colorstr('VisDroneDataset:')} Total 'person' instances after merge: {count_class_0 + count_class_1}")
        LOGGER.info(f"{colorstr('VisDroneDataset:')} Shifted {count_shifted} instances of other classes")
        
        return labels

In [11]:
from ultralytics.cfg import get_cfg
from ultralytics.utils import DEFAULT_CFG

DEFAULT_CFG

namespace(task='detect',
          mode='train',
          model=None,
          data=None,
          epochs=100,
          time=None,
          patience=100,
          batch=16,
          imgsz=640,
          save=True,
          save_period=-1,
          cache=False,
          device=None,
          workers=8,
          project=None,
          name=None,
          exist_ok=False,
          pretrained=True,
          optimizer='auto',
          verbose=True,
          seed=0,
          deterministic=True,
          single_cls=False,
          rect=False,
          cos_lr=False,
          close_mosaic=10,
          resume=False,
          amp=True,
          fraction=1.0,
          profile=False,
          freeze=None,
          multi_scale=False,
          overlap_mask=True,
          mask_ratio=4,
          dropout=0.0,
          val=True,
          split='val',
          save_json=False,
          save_hybrid=False,
          conf=None,
          iou=0.7,
          max_det=300,
    

In [12]:
from ultralytics.models.yolo.detect import DetectionTrainer
from ultralytics.utils.torch_utils import de_parallel

class CustomDetectionTrainer(DetectionTrainer):
    """Custom trainer that uses the VisDroneDataset."""
    
    def build_dataset(self, img_path, mode="train", batch=None):
        """Build VisDroneDataset instead of YOLODataset."""
        gs = max(int(de_parallel(self.model).stride.max() if self.model else 0), 32)
        return VisDroneDataset(
            img_path=img_path,
            imgsz=self.args.imgsz,
            batch_size=batch,
            augment=mode == "train",
            hyp=self.args,
            rect=mode == "val",
            cache=self.args.cache or None,
            single_cls=self.args.single_cls,
            stride=gs,
            pad=0.0 if mode == "train" else 0.5,
            prefix=colorstr(f"{mode}: "),
            task=self.args.task,
            data=self.data
        )
    
    def get_validator(self):
        """Returns a validator with adjusted class expectations."""
        validator = super().get_validator()
        # Add this one line to increase expected class count during validation
        validator.args.single_cls = True  # This works because it ignores class indices!
        return validator

In [14]:
df_train = top_5[['name', 'optimizer', 'lr0', 'lrf', 'momentum', 'weight_decay', 'cos_lr', 'box', 'cls', 'dfl']]

for idx, trial in df_train.iterrows():
    trial_config = config.copy()

    for param in top_5.columns:
        if param in trial and not pd.isna(trial[param]):
            trial_config['train'][param] = trial[param]

    # args = get_cfg(DEFAULT_CFG, overrides=trial_config['train'])
    
    trainer = CustomDetectionTrainer(overrides=trial_config['train'])
    model.names = trainer.data['names']

    results = trainer.train()

    save_results("../data/processed", config["train"]["name"], results)

    clear_cache()

Ultralytics 8.3.84 🚀 Python-3.10.12 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3080, 10002MiB)
engine/trainer: task=detect, mode=train, model=yolo12n, data=VisDrone.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=0, workers=8, project=EyeInTheSky, name=_tune_8290d_0001537, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=True, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=True, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

AttributeError: can't set attribute 'names'

In [ ]:
results

In [ ]:
# Check if this is the case
import torch
print(f"Allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
print(f"Cached: {torch.cuda.memory_reserved()/1e9:.2f} GB")

In [ ]:
# Force garbage collection
import gc
gc.collect()
torch.cuda.empty_cache()  # If using CUDA